# Productionize models with TFX (TensorFlow-Extended)

Training a machine learning model is only half of the journey. A model to be useful in the real-world you need to deploy it (i.e. productionize it). Productionization is the process of deploying the model on some infrastructure (e.g. a single machine or a cluster) and making it accessible for predicting via an web based API (e.g. REST).

<table align="left">
    <td>
        <a target="_blank" href="https://colab.research.google.com/github/thushv89/manning_tf2_in_action/blob/master/Ch14/14.1_Productionize_with_tfx.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
    </td>
</table>



In [1]:
import tensorflow as tf
#import tensorflow_hub as hub
import requests
print(tf.__version__)
import requests
import os
import time
import pandas as pd

assert_msg = "Pandas version should be < 1.0 for tfx"
assert pd.__version__.startswith('0.')

import random
import shutil
import tempfile
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import CSVLogger
import numpy as np
import pickle
from tensorflow.keras.models import load_model, Model
import glob
from functools import partial
import absl

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        print("Couldn't set memory_growth")
        pass
    
    
def fix_random_seed(seed):
    """ Setting the random seed of various libraries """
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed
random_seed=4321
fix_random_seed(random_seed)

2.4.2


## Caveats

* Pandas >= 1.0 will give,
  * `ValueError: Value must be a nonnegative integer or None` when displaying results (e.g. context.show() calls)
* Had to install apache-beam 2.29 (not 2.30) coz 2.30 was giving the error
  * `Attribute error: type StringMethods does not have attribute fullmatch`

# Download the data

We are going to use a dataset contining information about forest fires in the USA. The dataset is downloaded from [this link](http://archive.ics.uci.edu/ml/datasets/Forest+Fires). The dataset contains the following features.

* X - x-axis spatial coordinate within the Montesinho park map
* Y - y-axis spatial coordinate within the Montesinho park map
* month - month of the year
* day - day of the week
* FFMC - FFMC index from the FWI system (Fine Fuel Moisture Code (FFMC) represents fuel moisture of forest litter fuels under the shade of a forest canopy)
* DMC - DMC index from the FWI system (DMC is a numerical rating of the average moisture content of soil)
* DC - DC index from the FWI system (The Drought Code (DC) represents drying deep into the soil.)
* ISI - ISI index from the FWI system (The Initial Spread Index (ISI) is an expected rate of fire spread)
* temp - temperature in Celsius degrees
* RH - relative humidity in %
* wind - wind speed in km/h
* rain - outside rain in mm/m2
* area - the burned area of the forest (in ha)

In [12]:
# Downloading the data

import os
import requests
import tarfile

import shutil

# Retrieve the data
if not os.path.exists(os.path.join('data', 'csv', 'forestfires.csv')):
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists(os.path.join('data', 'csv')):
        os.makedirs(os.path.join('data', 'csv'))
    
    # Write to a file
    with open(os.path.join('data', 'csv', 'forestfires.csv'), 'wb') as f:
        f.write(r.content)
        
else:
    print("The forestfires.csv file already exists.")
    
    
if not os.path.exists(os.path.join('data', 'forestfires.names')):
    
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.names"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists('data'):
        os.makedirs('data')
    
    # Write to a file
    with open(os.path.join('data', 'forestfires.names'), 'wb') as f:
        f.write(r.content)
        
else:
    print("The forestfires.names file already exists.")



The forestfires.csv file already exists.
The forestfires.names file already exists.


## Separate train/valid and test datasets

In [13]:
import pandas as pd

df = pd.read_csv(os.path.join('data', 'csv', 'forestfires.csv'), index_col=None, header=0)
train_df = df.sample(frac=0.95, random_state=random_seed)
test_df = df.loc[~df.index.isin(train_df.index), :]

train_path = os.path.join('data','csv','train')
os.makedirs(train_path, exist_ok=True)
test_path = os.path.join('data','csv','test')
os.makedirs(test_path, exist_ok=True)

train_df.to_csv(os.path.join(train_path, 'forestfires.csv'), index=False, header=True)
test_df.to_csv(os.path.join(test_path, 'forestfires.csv'), index=False, header=True)

# Defining the pipeline root

In [14]:
# This is the directory containing the TFX Chicago Taxi Pipeline example.
_pipeline_root = os.path.join(os.getcwd(), 'pipeline', 'examples', 'forest_fires_pipeline')
print(_pipeline_root)

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline


## Defining an interactive context

Context runs various steps in the pipeline and maintains state between different steps. Interactive means that, it can be called/run via multiple calls, instead of doing via a single call. This is suited when running things on a notebook, where code is quite isolated.

In [15]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.

context = InteractiveContext(pipeline_name = "forest_fires", pipeline_root=_pipeline_root)

## Step 1: Define an Example Generator

This example generator generates examples (as `TFRecord` objects) from a CSV file. 

In [16]:
from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2

# You can change the configuration of generating train-eval split by passing a SplitConfig as below
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=3),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))

# Define the example generator
example_gen = CsvExampleGen(input_base=os.path.join('data', 'csv', 'train'))
# Run the example generator
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data data/csv/train/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:26223,xor_checksum:1625051115,sum_checksum:1625051115"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Difference channel and a pb2 (that's a protobuf)

https://www.tensorflow.org/tfx/api_docs/python/tfx/types/Channel (explain channels)

## Some results produced by `CsvExampleGen`

In [18]:
# Print outputs
print(example_gen.outputs)

# Accessing various variables in the example generator
artifact = example_gen.outputs['examples'].get()[0]

# Print some of that
print("Artifact split names: {}".format(artifact.split_names))
print("Artifact URI: {}".format(artifact.uri)


["train", "eval"] /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/CsvExampleGen/examples/1


## Print a few examples generated from the CSV file

In [19]:
import pprint

pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 2 records and decode them.
for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)


features {
  feature {
    key: "DC"
    value {
      float_list {
        value: 605.7999877929688
      }
    }
  }
  feature {
    key: "DMC"
    value {
      float_list {
        value: 141.3000030517578
      }
    }
  }
  feature {
    key: "FFMC"
    value {
      float_list {
        value: 95.0999984741211
      }
    }
  }
  feature {
    key: "ISI"
    value {
      float_list {
        value: 17.700000762939453
      }
    }
  }
  feature {
    key: "RH"
    value {
      int64_list {
        value: 43
      }
    }
  }
  feature {
    key: "X"
    value {
      int64_list {
        value: 5
      }
    }
  }
  feature {
    key: "Y"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "area"
    value {
      float_list {
        value: 2.0
      }
    }
  }
  feature {
    key: "day"
    value {
      bytes_list {
        value: "tue"
      }
    }
  }
  feature {
    key: "month"
    value {
      bytes_list {
        value: "aug"
    

## Step 2: Generate statistics of the data

Statistics compute various metrics. For, 
* continuous variables
  * like mean, standard deviation, count of zeros, median, min/max, etc.  
* categorical variables
  * count, missing count, number of unique values, etc.

In [20]:
from tfx.components import StatisticsGen

# Define the statistic generator
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
# Run the statistic  generator
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## View the statistics of train / eval data

In [21]:
context.show(statistics_gen.outputs['statistics'])

# Step 3: Infer the schema of the data

In [22]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Print `schema_gen` artifact location

In [23]:
print(schema_gen.outputs['schema'].artifacts.get()[0].uri)

/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/SchemaGen/schema/3


## Display the inferred schema

TFX uses the `tensorflow_data_validation` library to do that. To do this manually, do the following.

```
schema = tfdv.load_schema_text(
    os.path.join(schema_gen.outputs['schema']._artifacts[0].uri, 'schema.pbtxt')
) 
features_df, domain_df = tfdv.utils.display_util.get_schema_dataframe(schema)
tfdv.utils.display_util.display(domain_df)
```

In [24]:
import tensorflow_data_validation as tfdv

context.show(schema_gen.outputs['schema'])


,Type,Presence,Valency,Domain
Feature name,,,,
'DC',FLOAT,required,single,-
'DMC',FLOAT,required,single,-
'FFMC',FLOAT,required,single,-
'ISI',FLOAT,required,single,-
'RH',INT,required,single,-
'X',INT,required,single,-
'Y',INT,required,single,-
'area',FLOAT,required,single,-
'day',STRING,required,single,'day'


,Values
Domain,
'day',"'fri', 'mon', 'sat', 'sun', 'thu', 'tue', 'wed'"
'month',"'apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'oct', 'sep', 'nov'"


https://www.tensorflow.org/tfx/tutorials/tfx/components

## Step 4: Transforming data into features with `tensorflow_transform`

### Defining feature types

In [26]:
%%writefile forest_fires_constants.py

# Vocab based features

VOCAB_FEATURE_KEYS = ['day','month']

# Vocab based features are in fact categorical features
# Categorical features are assumed to each have a maximum value in the dataset
MAX_CATEGORICAL_FEATURE_VALUES = [7, 12]

# Dense features (these will go to the model as they are)

DENSE_FLOAT_FEATURE_KEYS = ['DC', 'DMC', 'FFMC', 'ISI', 'rain', 'temp', 'wind', 'X', 'Y']

# Bucketized features 

# Number of buckets used by tf.transform for encoding each feature.
BUCKET_FEATURE_BOUNDARIES = [(33, 66)]
BUCKET_FEATURE_KEYS = ['RH']

# Label features

LABEL_KEY = 'area'


def transformed_name(key):
    """ Define a function that will add a suffix to the feature name """
    
    return key + '_xf'


Overwriting forest_fires_constants.py


### Processing the columns to features

In [27]:
%%writefile forest_fires_transform.py

import tensorflow as tf
import tensorflow_transform as tft

import forest_fires_constants

_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BOUNDARIES = forest_fires_constants.BUCKET_FEATURE_BOUNDARIES
_LABEL_KEY = forest_fires_constants.LABEL_KEY
_transformed_name = forest_fires_constants.transformed_name


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
    
  # Treating dense features
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    # Preserve this feature as a dense float, setting nan's to the mean.
    outputs[_transformed_name(key)] = tft.scale_to_z_score(
        sparse_to_dense(inputs[key])
    )

  # Treating categorical features  
  for key in _VOCAB_FEATURE_KEYS:
    # Build a vocabulary for this feature.
    outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
        sparse_to_dense(inputs[key]),
        num_oov_buckets=1)

  # Treating bucketized features
  for key, boundary in zip(_BUCKET_FEATURE_KEYS, _BUCKET_FEATURE_BOUNDARIES):
    outputs[_transformed_name(key)] = tft.apply_buckets(
        sparse_to_dense(inputs[key]), bucket_boundaries=[boundary])

  # Treating the label feature
  outputs[_transformed_name(_LABEL_KEY)] = sparse_to_dense(inputs[_LABEL_KEY])

  return outputs

def sparse_to_dense(x):
    
    return tf.squeeze(
        tf.sparse.to_dense(tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1])),
        axis=1
    )


Overwriting forest_fires_transform.py


### Running the transform step

In [28]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('forest_fires_transform.py'),
)

context.run(transform)


INFO:absl:Generating ephemeral wheel package for '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/forest_fires_transform.py' (including modules: ['forest_fires_constants', 'forest_fires_transform', 'forest_fires_trainer']).
INFO:absl:User module package has hash fingerprint version 94f1c4f6e276f7f35017a995dbf8b4cfb19eccc404a37a70a26808bcede2daab.
INFO:absl:Executing: ['/home/thushv89/anaconda3/envs/manning.tf2.tfx/bin/python', '/tmp/tmp0su_q6t8/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpgey8o2mx', '--dist-dir', '/tmp/tmpxi6lpny7']
INFO:absl:Successfully built user code wheel distribution at '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Transform-0.0+94f1c4f6e276f7f35017a995dbf8b4cfb19eccc404a37a70a26808bcede2daab-py3-none-any.whl'; target user module is 'forest_fires_transform'.
INFO:absl:Full user module path is 'forest_fires_transform@/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipelin

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Y has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature rain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature wind has no shape. Setting to VarLenSparseTensor.


INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Y has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature rain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature wind has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTen

...

INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/4/.temp_path/tftransform_tmp/9a8db372cef049ecab0e4b3ae1c28641/assets


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/4/.temp_path/tftransform_tmp/e6ad3eca3ef14df5a47150c231e605b3/assets


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/4/.temp_path/tftransform_tmp/e6ad3eca3ef14df5a47150c231e605b3/assets


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 4
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/4"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 5
        type_id: 5
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transformed_examples/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 6
        type_id: 12
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/updated_analyzer_cache/4"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 12
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Print a few examples generated from the transformer

In [30]:
import pprint

import forest_fires_constants

_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_LABEL_KEY = forest_fires_constants.LABEL_KEY
pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.

example_records = []
float_features = [f+'_xf' for f in _DENSE_FLOAT_FEATURE_KEYS + ['area']]
int_features = [f+'_xf' for f in _BUCKET_FEATURE_KEYS + _VOCAB_FEATURE_KEYS]
for tfrecord in dataset.skip(200).take(5):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  record = [example.features.feature[f].int64_list.value for f in int_features] + [example.features.feature[f].float_list.value for f in float_features]
  example_records.append(record)
  print(example)
  print("="*50)

features {
  feature {
    key: "DC_xf"
    value {
      float_list {
        value: 0.4196213185787201
      }
    }
  }
  feature {
    key: "DMC_xf"
    value {
      float_list {
        value: 0.9436285495758057
      }
    }
  }
  feature {
    key: "FFMC_xf"
    value {
      float_list {
        value: 0.8787775039672852
      }
    }
  }
  feature {
    key: "ISI_xf"
    value {
      float_list {
        value: 1.5760188102722168
      }
    }
  }
  feature {
    key: "RH_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "X_xf"
    value {
      float_list {
        value: -0.2647276520729065
      }
    }
  }
  feature {
    key: "Y_xf"
    value {
      float_list {
        value: 0.54094398021698
      }
    }
  }
  feature {
    key: "area_xf"
    value {
      float_list {
        value: 2.7699999809265137
      }
    }
  }
  feature {
    key: "day_xf"
    value {
      int64_list {
        value: 3
      }
    }
  }
  feature {

### Presenting as a dataframe

In [31]:
sample_df = pd.DataFrame.from_records(example_records, columns=int_features+float_features)
sample_df.head()

,RH_xf,day_xf,month_xf,DC_xf,DMC_xf,FFMC_xf,ISI_xf,rain_xf,temp_xf,wind_xf,X_xf,Y_xf,area_xf
0,[0],[3],[0],[0.4196213185787201],[0.9436285495758057],[0.8787775039672852],[1.5760188102722168],[-0.08982300013303757],[2.3198556900024414],[-0.47125762701034546],[-0.2647276520729065],[0.54094398021698],[2.7699999809265137]
1,[1],[1],[1],[0.5483551025390625],[-0.4714054465293884],[0.5791595578193665],[1.3903242349624634],[-0.08982300013303757],[0.07178180664777756],[0.8417152166366577],[-0.6947485208511353],[-0.23549708724021912],[0.0]
2,[1],[3],[1],[0.850985050201416],[-0.10668977349996567],[0.15338461101055145],[-0.6110513806343079],[-0.08982300013303757],[0.5811113119125366],[-0.9850294589996338],[1.4553557634353638],[1.3173850774765015],[7.480000019073486]
3,[0],[4],[1],[0.6078682541847229],[0.34098687767982483],[0.3583875000476837],[0.007930779829621315],[-0.08982300013303757],[1.4592652320861816],[-0.9850294589996338],[-0.2647276520729065],[-1.011938214302063],[0.0]
4,[1],[1],[1],[0.7720564007759094],[-0.2538282871246338],[0.23223169147968292],[0.0904618501663208],[-0.08982300013303757],[0.24741259217262268],[0.5562863945960999],[-0.2647276520729065],[-0.23549708724021912],[1.2599999904632568]


## Step 5: Defining the Trainer

In [33]:
%%writefile forest_fires_trainer.py

# Classif vs regression signature defs
# https://www.tensorflow.org/tfx/serving/signature_defs

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

import forest_fires_constants


_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BOUNDARIES = forest_fires_constants.BUCKET_FEATURE_BOUNDARIES
_MAX_CATEGORICAL_FEATURE_VALUES = forest_fires_constants.MAX_CATEGORICAL_FEATURE_VALUES
_LABEL_KEY = forest_fires_constants.LABEL_KEY
_transformed_name = forest_fires_constants.transformed_name


def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]


def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""

  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function
  def serve_tf_examples_fn(serialized_tf_examples):
    """Returns the output to be used in the serving signature."""
    feature_spec = tf_transform_output.raw_feature_spec()
    feature_spec.pop(_LABEL_KEY)
    parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
    transformed_features = model.tft_layer(parsed_features)
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema)


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying taxi data.

  Args:
    hidden_units: [int], the layer sizes of the DNN (input layer first).

  Returns:
    A keras Model.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]
  
  categorical_columns = [
      tf.feature_column.indicator_column(
          tf.feature_column.categorical_column_with_identity(
              key, num_buckets=len(boundaries)+1
          )
      ) for key, boundaries in zip(
          _transformed_names(_BUCKET_FEATURE_KEYS), _BUCKET_FEATURE_BOUNDARIES
      )
  ]
    
  categorical_columns += [
      tf.feature_column.indicator_column(
          tf.feature_column.categorical_column_with_identity( 
              key,
              num_buckets=num_buckets,
              default_value=0
          ) for key, num_buckets in zip(
              _transformed_names(_VOCAB_FEATURE_KEYS),
              _MAX_CATEGORICAL_FEATURE_VALUES
          )
      )
  ]

  model = _dnn_regressor(
      columns=real_valued_columns+categorical_columns,
      dnn_hidden_units=[128, 64])
  return model


def _dnn_regressor(columns, dnn_hidden_units):
  """Build a simple keras wide and deep model.

  Args:
    wide_columns: Feature columns wrapped in indicator_column for wide (linear)
      part of the model.
    deep_columns: Feature columns for deep part of the model.
    dnn_hidden_units: [int], the layer sizes of the hidden DNN.

  Returns:
    A Wide and Deep Keras model
  """
  # Following values are hard coded for simplicity in this example,
  # However prefarably they should be passsed in as hparams.

  # Keras needs the feature definitions at compile time.  
  input_layers = {
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype=tf.float32)
      for colname in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  }
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_VOCAB_FEATURE_KEYS)
  })
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_BUCKET_FEATURE_KEYS)
  })  

  # TODO(b/161952382): Replace with Keras preprocessing layers.
  output = tf.keras.layers.DenseFeatures(columns)(input_layers)
  for numnodes in dnn_hidden_units:
    output = tf.keras.layers.Dense(numnodes, activation='tanh')(output)
  
  output = tf.keras.layers.Dense(1)(output)

  model = tf.keras.Model(input_layers, output)
  model.compile(
      loss='mean_squared_error',
      optimizer=tf.keras.optimizers.Adam(lr=0.001)
  )
  model.summary(print_fn=absl.logging.info)

  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, 40)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, 40)

  model = _build_keras_model()
  
  os.makedirs(os.path.join('data','model_performance'), exist_ok=True)

  csv_callback = tf.keras.callbacks.CSVLogger(
    os.path.join('data', 'model_performance', 'model1.csv'), append=False
  )

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      epochs=10,
      callbacks=[csv_callback]
  )

  signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(
              model, tf_transform_output
          ).get_concrete_function(
              tf.TensorSpec(
                  shape=[None],
                  dtype=tf.string,
                  name='examples'
              )
          ),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Overwriting forest_fires_trainer.py


### Understanding `tf.feature_column`

In [32]:
# https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column

a = tf.feature_column.numeric_column("a", shape=())
x = tf.keras.layers.DenseFeatures(a)({'a': [0.5, 0.6]})
print(x)

b = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_identity('b', num_buckets=10))
y = tf.keras.layers.DenseFeatures(b)({'b': [5, 2]})
print(y)



tf.Tensor(
[[0.5]
 [0.6]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]], shape=(2, 10), dtype=float32)


### Running the trainer

In [34]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2
import tensorflow.keras.backend as K

K.clear_session()


trainer = Trainer(
    module_file=os.path.abspath("forest_fires_trainer.py"),
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=100))
context.run(trainer)


INFO:absl:Generating ephemeral wheel package for '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/forest_fires_trainer.py' (including modules: ['forest_fires_constants', 'forest_fires_transform', 'forest_fires_trainer']).
INFO:absl:User module package has hash fingerprint version e9b2124efef7bb3e78d8fc035e602e1a6cfb433e93e5c0bf0eb3d1a6c5a80539.
INFO:absl:Executing: ['/home/thushv89/anaconda3/envs/manning.tf2.tfx/bin/python', '/tmp/tmpk7ywtjz5/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmplmvolul7', '--dist-dir', '/tmp/tmpy2kfbenu']
INFO:absl:Successfully built user code wheel distribution at '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Trainer-0.0+e9b2124efef7bb3e78d8fc035e602e1a6cfb433e93e5c0bf0eb3d1a6c5a80539-py3-none-any.whl'; target user module is 'forest_fires_trainer'.
INFO:absl:Full user module path is 'forest_fires_trainer@/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/exampl

Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x7fc6c401fc50>>
Traceback (most recent call last):
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/home/thushv89/anaconda3/envs/manning.t

    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 2969, in _get_concrete_function_internal_garbage_collected
    graph_function, _ = self._maybe_define_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 3361, in _maybe_define_function
    graph_function = self._create_graph_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/functi

    graph_function = self._create_graph_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 3206, in _create_graph_function
    capture_by_value=self._capture_by_value),
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/framework/func_graph.py", line 990, in func_graph_from_py_func
    func_outputs = python_func(*func_args, **func_kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 634, in wrapped_fn
    out = weak_wrapped_fn().__wrapped__(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/saved_model/function_deserialization.py", line 253, in restored_function_body
    return _call_concrete_function(function, inputs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packag

INFO:absl:Feature area_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature day_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature month_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature rain_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature temp_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature wind_xf has a shape . Setting to DenseTensor.
INFO:absl:Model: "model"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:DC_xf (InputLayer)              [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:DMC_xf (InputLayer)             [(No

Epoch 1/10
100/100 [==============================] - ETA: 1:44 - loss: 43841.199 - ETA: 0s - loss: 7717.7549  - ETA: 0s - loss: 6834.36 - 2s 5ms/step - loss: 6663.7251 - val_loss: 541.6857
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 1038.04 - ETA: 0s - loss: 5822.72 - ETA: 0s - loss: 5846.90 - 0s 3ms/step - loss: 5854.0528 - val_loss: 551.4136
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 859.254 - ETA: 0s - loss: 6011.84 - ETA: 0s - loss: 5880.66 - 0s 3ms/step - loss: 5860.2029 - val_loss: 557.9216
Epoch 4/10
100/100 [==============================] - ETA: 0s - loss: 1043.58 - ETA: 0s - loss: 5807.06 - ETA: 0s - loss: 5800.36 - 0s 3ms/step - loss: 5807.0441 - val_loss: 551.7376
Epoch 5/10
100/100 [==============================] - ETA: 0s - loss: 735.089 - ETA: 0s - loss: 5077.97 - ETA: 0s - loss: 5362.05 - 0s 3ms/step - loss: 5419.9164 - val_loss: 550.9117
Epoch 6/10
100/100 [==============================] - ETA: 0s - loss: 1394.08 

INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/5/Format-Serving/assets


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/5/Format-Serving/assets
INFO:absl:Training complete. Model written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/5/Format-Serving. ModelRun written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model_run/5
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 5
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/5"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 8
        type_id: 15
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model_run/5"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Evaluate the trained model

### Load the model

In [47]:
model = load_model(os.path.join(trainer.outputs["model"]._artifacts[0].uri, "Format-Serving"))
model.summary()


Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fc5c9e2c7b8> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fc585484860>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fc5c9e2c7b8> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fc585484860>).


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DC_xf (InputLayer)              [(None,)]            0                                            
__________________________________________________________________________________________________
DMC_xf (InputLayer)             [(None,)]            0                                            
__________________________________________________________________________________________________
FFMC_xf (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
ISI_xf (InputLayer)             [(None,)]            0                                            
______________________________________________________________________________________________

### Predict with the loaded model

In [123]:
#https://www.tensorflow.org/tfx/transform/api_docs/python/tft/TFTransformOutput
    
import tensorflow_transform as tft
tf_transform_output = tft.TFTransformOutput(transform.outputs["transform_graph"]._artifacts[0].uri)

test_df = pd.read_csv(os.path.join(test_path, 'forestfires.csv'), index_col=None, header=0)

sample = {}
for col, val in test_df.to_dict(orient='list').items():
    
    if col=='area':
        continue
    
    # Transformation came from https://github.com/tensorflow/transform/blob/d401253873a2079c245049572e8488347adb3cd4/examples/census_example_v2.py#L92
    if isinstance(val[0], int):
        sample[col] = tf.RaggedTensor.from_tensor(tf.cast(tf.expand_dims(val, -1), dtype='int64')).to_sparse()
        
    else:
        sample[col] = tf.RaggedTensor.from_tensor(tf.expand_dims(val, -1)).to_sparse()
        
print(sample)
print(tf_transform_output.raw_feature_spec()) 



col X
col Y
col RH
{'X': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fc567eae8d0>, 'Y': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fc56c06fc18>, 'month': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fc567eaebe0>, 'day': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fc567eaec18>, 'FFMC': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fc567eaeb38>, 'DMC': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fc567eaeba8>, 'DC': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fc567eae978>, 'ISI': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fc567eae9e8>, 'temp': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fc567eae940>, 'RH': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fc567eaea90>, 'wind': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fc567

In [124]:
transformed_features = tf_transform_output.transform_features_layer()(sample)

In [125]:
model.predict(transformed_features)

array([[18.647676  ],
       [ 9.986312  ],
       [36.474617  ],
       [ 8.55777   ],
       [ 6.008959  ],
       [13.074179  ],
       [ 7.7907286 ],
       [ 0.61505735],
       [12.601279  ],
       [-1.1718907 ],
       [17.229527  ],
       [ 1.711335  ],
       [21.89656   ],
       [10.260798  ],
       [13.469515  ],
       [30.931496  ],
       [ 9.8805    ],
       [ 0.8249037 ],
       [11.299833  ],
       [-3.8344867 ],
       [ 8.084992  ],
       [14.180465  ],
       [ 5.6950426 ],
       [11.228131  ],
       [10.43844   ],
       [ 6.661993  ]], dtype=float32)

## Adapting the schema and treating anomalies in the data

In [126]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'DC',FLOAT,required,single,-
'DMC',FLOAT,required,single,-
'FFMC',FLOAT,required,single,-
'ISI',FLOAT,required,single,-
'RH',INT,required,single,-
'X',INT,required,single,-
'Y',INT,required,single,-
'area',FLOAT,required,single,-
'day',STRING,required,single,'day'


,Values
Domain,
'day',"'fri', 'mon', 'sat', 'sun', 'thu', 'tue', 'wed'"
'month',"'apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'oct', 'sep', 'nov'"


### Adapting the schema to identify anomalies

These various attributes are defined at `https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto`

In [129]:
# Use this for detecting what anomly conditions are detected
#https://www.tensorflow.org/tfx/data_validation/anomalies
    
schema = tfdv.load_schema_text(os.path.join(schema_gen.outputs['schema']._artifacts[0].uri, 'schema.pbtxt'))
statistics = tfdv.generate_statistics_from_csv(os.path.join('data', 'csv', 'train', 'forestfires.csv'))

y_feature = tfdv.get_feature(schema, 'Y')
isi_feature = tfdv.get_feature(schema, 'ISI')
ffmc_feature = tfdv.get_feature(schema, 'FFMC')
rain_feature = tfdv.get_feature(schema, 'rain')
area_feature = tfdv.get_feature(schema, 'area')

y_feature.int_domain.max = 7
isi_feature.float_domain.max = 30.0
ffmc_feature.float_domain.min = 60.0
rain_feature.float_domain.max = 1.0
area_feature.float_domain.max = 75.0


anomalies = tfdv.validate_statistics(statistics = statistics, schema = schema)
#anomalies = tfdv.validate_examples_in_csv(os.path.join('data', 'csv', 'forestfires.csv'))

tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'rain',Out-of-range values,Unexpectedly high value: 6.4>1(upto six significant digits)
'FFMC',Out-of-range values,Unexpectedly low values: 18.7<60(upto six significant digits)
'area',Out-of-range values,Unexpectedly high value: 1090.84>75(upto six significant digits)
'Y',Out-of-range values,Unexpectedly large value: 9.
'ISI',Out-of-range values,Unexpectedly high value: 56.1>30(upto six significant digits)


## Overwrite the schema

In [131]:
tfdv.write_schema_text(schema, os.path.join(schema_gen.outputs['schema']._artifacts[0].uri, 'schema.pbtxt'))

## Checking examples

In [132]:
from tfx.components import ExampleValidator


example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

context.run(example_validator)


INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/ExampleValidator/anomalies/9/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/ExampleValidator/anomalies/9/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 9
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 12
        type_id: 23
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/ExampleValidator/anomalies/9"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### View anomalies

In [133]:
context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'rain',Out-of-range values,Unexpectedly high value: 6.4>1(upto six significant digits)
'ISI',Out-of-range values,Unexpectedly high value: 56.1>30(upto six significant digits)
'Y',Out-of-range values,Unexpectedly large value: 9.
'area',Out-of-range values,Unexpectedly high value: 1090.84>75(upto six significant digits)
'FFMC',Out-of-range values,Unexpectedly low values: 18.7<60(upto six significant digits)


,Anomaly short description,Anomaly long description
Feature name,,
'Y',Out-of-range values,Unexpectedly large value: 9.
'area',Out-of-range values,Unexpectedly high value: 212.88>75(upto six significant digits)


### Remove anomalies

In [2]:
df = pd.read_csv(os.path.join('data','csv', 'train','forestfires.csv'), header=0)

print("Original shape: {}".format(df.shape))
valid_df = df[(df["FFMC"]>=60)&(df["ISI"]<=30)&(df["Y"]<=7)&(df["area"]<=75)&(df["rain"]<=1)]
print("After anomalies removed shape: {}".format(valid_df.shape))
os.makedirs(os.path.join('data','csv_valid'),exist_ok=True)
valid_df.to_csv(os.path.join('data','csv_valid','forestfires.csv'), header=True, index=False)

NameError: name 'pd' is not defined

### View statistics of data side by side (before and after removing anomalies)

In [136]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=3),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))


example_gen = CsvExampleGen(input_base=os.path.join('data', 'csv_valid'))
context.run(example_gen)

from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

context.show(statistics_gen.outputs['statistics'])

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:Generating examples.
INFO:absl:Processing input csv data data/csv_valid/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipelin

## Transform data (after removing anomalies)

In [137]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('forest_fires_transform.py'))

context.run(transform)

INFO:absl:Generating ephemeral wheel package for '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/forest_fires_transform.py' (including modules: ['forest_fires_constants', 'forest_fires_transform', 'forest_fires_trainer']).
INFO:absl:User module package has hash fingerprint version e9b2124efef7bb3e78d8fc035e602e1a6cfb433e93e5c0bf0eb3d1a6c5a80539.
INFO:absl:Executing: ['/home/thushv89/anaconda3/envs/manning.tf2.tfx/bin/python', '/tmp/tmp33uhsd8m/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp6ixj1uuv', '--dist-dir', '/tmp/tmpnpopy9ai']
INFO:absl:Successfully built user code wheel distribution at '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Transform-0.0+e9b2124efef7bb3e78d8fc035e602e1a6cfb433e93e5c0bf0eb3d1a6c5a80539-py3-none-any.whl'; target user module is 'forest_fires_transform'.
INFO:absl:Full user module path is 'forest_fires_transform@/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipelin

INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Y has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature rain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature wind has no shape. Setting to VarLenSparseTensor.


Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x7fc56c67e080>>
Traceback (most recent call last):
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/home/thushv89/anaconda3/envs/manning.t

    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 2969, in _get_concrete_function_internal_garbage_collected
    graph_function, _ = self._maybe_define_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 3361, in _maybe_define_function
    graph_function = self._create_graph_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/functi

    graph_function = self._create_graph_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 3206, in _create_graph_function
    capture_by_value=self._capture_by_value),
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/framework/func_graph.py", line 990, in func_graph_from_py_func
    func_outputs = python_func(*func_args, **func_kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 634, in wrapped_fn
    out = weak_wrapped_fn().__wrapped__(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/saved_model/function_deserialization.py", line 253, in restored_function_body
    return _call_concrete_function(function, inputs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packag

...

Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x7fc7344e0d68>>
Traceback (most recent call last):
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/home/thushv89/anaconda3/envs/manning.t

INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/12/.temp_path/tftransform_tmp/d19b28b41b9d4e91b4b98986fce4a266/assets


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/12/.temp_path/tftransform_tmp/d19b28b41b9d4e91b4b98986fce4a266/assets


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 12
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 15
        type_id: 11
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transform_graph/12"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 16
        type_id: 5
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/transformed_examples/12"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 17
        type_id: 12
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Transform/updated_analyzer_cache/12"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 12
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Train the model (after removing anomalies)

In [138]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=os.path.abspath("forest_fires_trainer.py"),
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=100))
context.run(trainer)


INFO:absl:Generating ephemeral wheel package for '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/forest_fires_trainer.py' (including modules: ['forest_fires_constants', 'forest_fires_transform', 'forest_fires_trainer']).
INFO:absl:User module package has hash fingerprint version e9b2124efef7bb3e78d8fc035e602e1a6cfb433e93e5c0bf0eb3d1a6c5a80539.
INFO:absl:Executing: ['/home/thushv89/anaconda3/envs/manning.tf2.tfx/bin/python', '/tmp/tmpccrvivyf/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpre5osik4', '--dist-dir', '/tmp/tmp588rc9vh']
INFO:absl:Successfully built user code wheel distribution at '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/_wheels/tfx_user_code_Trainer-0.0+e9b2124efef7bb3e78d8fc035e602e1a6cfb433e93e5c0bf0eb3d1a6c5a80539-py3-none-any.whl'; target user module is 'forest_fires_trainer'.
INFO:absl:Full user module path is 'forest_fires_trainer@/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/exampl

INFO:absl:__________________________________________________________________________________________________
INFO:absl:day_xf (InputLayer)             [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:month_xf (InputLayer)           [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:rain_xf (InputLayer)            [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:temp_xf (InputLayer)            [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:wind_xf (

Epoch 1/10
100/100 [==============================] - ETA: 34s - loss: 77.206 - ETA: 0s - loss: 154.727 - ETA: 0s - loss: 151.227 - 1s 4ms/step - loss: 149.5874 - val_loss: 140.5322
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 146.553 - ETA: 0s - loss: 142.895 - ETA: 0s - loss: 137.696 - 0s 3ms/step - loss: 136.7332 - val_loss: 142.5079
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 50.43 - ETA: 0s - loss: 120.883 - ETA: 0s - loss: 124.200 - 0s 3ms/step - loss: 124.8875 - val_loss: 145.2618
Epoch 4/10
100/100 [==============================] - ETA: 0s - loss: 174.504 - ETA: 0s - loss: 132.507 - ETA: 0s - loss: 129.327 - 0s 3ms/step - loss: 128.8983 - val_loss: 147.7562
Epoch 5/10
100/100 [==============================] - ETA: 0s - loss: 69.70 - ETA: 0s - loss: 114.987 - ETA: 0s - loss: 117.223 - 0s 3ms/step - loss: 117.6053 - val_loss: 149.0715
Epoch 6/10
100/100 [==============================] - ETA: 0s - loss: 165.699 - ETA: 0s - loss

Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x7fc565358f28>>
Traceback (most recent call last):
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py", line 208, in __del__
    self._destroy_resource()
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/home/thushv89/anaconda3/envs/manning.t

    result = self._call(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 726, in _initialize
    *args, **kwds))
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 2969, in _get_concrete_function_internal_garbage_collected
    graph_function, _ = self._maybe_define_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 3361, in _maybe_define_function
    graph_function = self._create_graph_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/functi

    graph_function = self._create_graph_function(args, kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 3206, in _create_graph_function
    capture_by_value=self._capture_by_value),
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/framework/func_graph.py", line 990, in func_graph_from_py_func
    func_outputs = python_func(*func_args, **func_kwargs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 634, in wrapped_fn
    out = weak_wrapped_fn().__wrapped__(*args, **kwds)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packages/tensorflow/python/saved_model/function_deserialization.py", line 253, in restored_function_body
    return _call_concrete_function(function, inputs)
  File "/home/thushv89/anaconda3/envs/manning.tf2.tfx/lib/python3.6/site-packag

INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/13/Format-Serving/assets


INFO:tensorflow:Assets written to: /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/13/Format-Serving/assets
INFO:absl:Training complete. Model written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/13/Format-Serving. ModelRun written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model_run/13
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 13
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 18
        type_id: 14
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/13"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 19
        type_id: 15
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model_run/13"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [35]:
# bind mount docker.sock
# urllib3.exceptions.ProtocolError: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))
# Add the docker group to container
# urllib3.exceptions.ProtocolError: ('Connection aborted.', PermissionError(13, 'Permission denied'))

In [36]:
# Defining pipelines
# https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple

# Understand different objects here 
# https://github.com/tensorflow/tfx/blob/master/tfx/proto/infra_validator.proto

# https://github.com/tensorflow/tfx/issues/2912

# TFX serve api
# https://www.tensorflow.org/tfx/serving/api_rest

# if api call for predic tnot in correct format
# "error": "Failed to process element: 0 key: X of 'instances' list. Error: Invalid argument: JSON object: does not have named input: X"

from tfx.components import InfraValidator
from tfx.proto import infra_validator_pb2
infra_validator = InfraValidator(
    model=trainer.outputs['model'],
    # This is the source for the data that will be used to build a request.
    examples=example_gen.outputs['examples'],
    serving_spec=infra_validator_pb2.ServingSpec(
        # Depending on what kind of model server you're using, RequestSpec
        # should specify the compatible one.
        tensorflow_serving=infra_validator_pb2.TensorFlowServing(tags=['2.4.1-gpu']),
        local_docker=infra_validator_pb2.LocalDockerConfig(
            client_base_url="unix:///var/run/docker.sock", client_api_version="1.40"
        ),
    ),
    request_spec=infra_validator_pb2.RequestSpec(
        # InfraValidator will look at how "classification" signature is defined
        # in the model, and automatically convert some samples from `examples`
        # artifact to prediction RPC requests.
        tensorflow_serving=infra_validator_pb2.TensorFlowServingRequestSpec(
            signature_names=['serving_default']
        ),
        num_examples=5  # How many requests to make.
    )
)

context.run(infra_validator)

INFO:absl:Running driver for InfraValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for InfraValidator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:InfraValidator will be run in LOAD_AND_QUERY mode.
INFO:absl:tag_set is not given. Using {'serve'} instead.
INFO:absl:Creating temp directory at /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/.temp/6/
INFO:absl:Starting infra validation (attempt 1/5).
INFO:absl:Starting LocalDockerRunner(image: tensorflow/serving:2.4.1-gpu).
INFO:absl:Running container with parameter {'auto_remove': True, 'detach': True, 'publish_all_ports': True, 'image': 'tensorflow/serving:2.4.1-gpu', 'environment': {'MODEL_NAME': 'infra-validation-model', 'MODEL_BASE_PATH': '/model'}, 'mounts': [{'Target': '/model/infra-validation-model/1', 'Source': '/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/.temp/6/i

ExecutionResult(
    component_id: InfraValidator
    execution_id: 6
    outputs:
        blessing: Channel(
            type_name: InfraBlessing
            artifacts: [Artifact(artifact: id: 9
        type_id: 17
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/InfraValidator/blessing/6"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "InfraValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "InfraBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [37]:
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and 
        # remove the label_key.
        tfma.ModelSpec(label_key='area')
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='MeanSquaredError',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          upper_bound={'value': 7500.0}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.LOWER_IS_BETTER,
                          absolute={'value': 1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        tfma.SlicingSpec(feature_keys=['month'])
    ])


In [38]:
from tfx import v1 as tfx

model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

context.run(model_resolver)



INFO:absl:Running driver for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [39]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator)


INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
ERROR:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "MeanSquaredError",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 1e-10,\n              "direction": "LOWER_IS_BETTER"\n            },\n            "value_threshold": {\n              "upper_bound": 7500.0\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "area"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "month"\n      ]\n    }\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': None, 'example_splits': 'null', 'module_file': None, '


Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fc630288d30> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fc6307a6da0>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fc630288d30> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fc6307a6da0>).


INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
ERROR:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "MeanSquaredError",\n          "threshold": {\n            "change_threshold": {\n              "absolute": 1e-10,\n              "direction": "LOWER_IS_BETTER"\n            },\n            "value_threshold": {\n              "upper_bound": 7500.0\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "area"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "month"\n      ]\n    }\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': None, 'example_splits': 'null', 'module_file': None, 'module_path': None} 'custom_extractors'
INFO:absl:Request was made to ignore the baseline ModelSpec and a


Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fc6c2f64ef0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fc7346a4128>).


...


Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fc5c8201588> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fc5c81f95c0>).


INFO:absl:Evaluation complete. Results written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/evaluation/8.
INFO:absl:Checking validation results.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result True written to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/blessing/8.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 10
        type_id: 20
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/evaluation/8"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 11
        type_id: 21
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/blessing/8"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Trainer/model/5"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 7
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 21
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [38]:
print(evaluator.outputs.blessing.get()[0].uri)

/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Evaluator/blessing/11


In [66]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))


metric_validations_per_slice {
  slice_key {
  }
  failures {
    metric_key {
      name: "mean_squared_error"
    }
    metric_threshold {
      value_threshold {
        upper_bound {
          value: 200.0
        }
      }
    }
    metric_value {
      double_value {
        value: 846.5555419921875
      }
    }
  }
}
metric_validations_per_slice {
  slice_key {
    single_slice_keys {
      column: "month"
      bytes_value: "mar"
    }
  }
  failures {
    metric_key {
      name: "mean_squared_error"
    }
    metric_threshold {
      value_threshold {
        upper_bound {
          value: 200.0
        }
      }
    }
    metric_value {
      double_value {
        value: 217.6882781982422
      }
    }
  }
}
metric_validations_per_slice {
  slice_key {
    single_slice_keys {
      column: "month"
      bytes_value: "aug"
    }
  }
  failures {
    metric_key {
      name: "mean_squared_error"
    }
    metric_threshold {
      value_threshold {
        upper_bound {
     

In [ ]:
# curl http://localhost:8501/v1/models/forest_fires_model
# curl -H "Content-Type: application/json" --data @data/request/request_body_regress.json http://localhost:8501/v1/models/forest_fires_mode:regress

{
  "signature_name": "serving_default",

  "context": {
    "X": 7,
    "Y": 4
  },

  "examples": [
    {
        "month": "oct",
        "day": "fri",
        "FFMC": 60.0,
        "DMC": 30.0,
        "DC": 200.0,
        "ISI": 9.0,
        "temp": 30.0,
        "RH": 50,
        "wind": 10.0,
        "rain": 0.0,
    },
    {
        "month": "jan",
        "day": "fri",
        "FFMC": 80.0,
        "DMC": 30.0,
        "DC": 200.0,
        "ISI": 9.0,
        "temp": 15.0,
        "RH": 50,
        "wind": 1.0,
        "rain": 1.0,
    }    
  ]
}


In [37]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
  model=trainer.outputs['model'],
  model_blessing=evaluator.outputs['blessing'],
  infra_blessing=infra_validator.outputs['blessing'],
  push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory=os.path.join('test_push'))
  )
)
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:Model version: 1624775221
INFO:absl:Model written to serving path test_push/1624775221.
INFO:absl:Model pushed to /home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Pusher/pushed_model/12.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 12
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 15
        type_id: 23
        uri: "/home/thushv89/code/manning_tf2_in_action/Ch14/tfx/pipeline/examples/forest_fires_pipeline/Pusher/pushed_model/12"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "test_push/1624775221"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1624775221"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Appendix: Defining a model using the TF Estimator API


```
%%writefile forest_fires_trainer.py

import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils
from tfx_bsl.tfxio import dataset_options

import forest_fires_constants

_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BOUNDARIES = forest_fires_constants.BUCKET_FEATURE_BOUNDARIES
_MAX_CATEGORICAL_FEATURE_VALUES = forest_fires_constants.MAX_CATEGORICAL_FEATURE_VALUES
_LABEL_KEY = forest_fires_constants.LABEL_KEY
_transformed_name = forest_fires_constants.transformed_name


def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]


# Tf.Transform considers these features as "raw"
def _get_raw_feature_spec(schema):
  return schema_utils.schema_as_feature_spec(schema).feature_spec


def _build_estimator(config, hidden_units):
  """Build an estimator for predicting the tipping behavior of taxi riders.
  Args:
    config: tf.estimator.RunConfig defining the runtime environment for the
      estimator (including model_dir).
    hidden_units: [int], the layer sizes of the DNN (input layer first)
    warm_start_from: Optional directory to warm start from.
  Returns:
    A dict of the following:

      - estimator: The estimator that will be used for training and eval.
      - train_spec: Spec for training.
      - eval_spec: Spec for eval.
      - eval_input_receiver_fn: Input function for eval.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]
  
  categorical_columns = [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=len(boundaries)+1
      ) for key, boundaries in zip(
          _transformed_names(_BUCKET_FEATURE_KEYS), _BUCKET_FEATURE_BOUNDARIES
      )
  ]
    
  categorical_columns += [
      tf.feature_column.categorical_column_with_identity(  # pylint: disable=g-complex-comprehension
          key,
          num_buckets=num_buckets,
          default_value=0) for key, num_buckets in zip(
              _transformed_names(_VOCAB_FEATURE_KEYS),
              _MAX_CATEGORICAL_FEATURE_VALUES)
  ]

  return tf.estimator.DNNLinearCombinedRegressor(
      config=config,
      linear_feature_columns=categorical_columns,
      dnn_feature_columns=real_valued_columns,
      dnn_hidden_units=hidden_units,
      dnn_activation_fn=tf.nn.leaky_relu,
      linear_optimizer=tf.keras.optimizers.RMSprop(lr=0.0001)
      )


def _example_serving_receiver_fn(tf_transform_graph, schema):
  """Build the serving in inputs.
  Args:
    tf_transform_graph: A TFTransformOutput.
    schema: the schema of the input data.
  Returns:
    Tensorflow graph which parses examples, applying tf-transform to them.
  """
  raw_feature_spec = _get_raw_feature_spec(schema)
  raw_feature_spec.pop(_LABEL_KEY)

  raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
      raw_feature_spec, default_batch_size=None)
  serving_input_receiver = raw_input_fn()

  transformed_features = tf_transform_graph.transform_raw_features(
      serving_input_receiver.features)

  return tf.estimator.export.ServingInputReceiver(
      transformed_features, serving_input_receiver.receiver_tensors)


def _eval_input_receiver_fn(tf_transform_graph, schema):
  """Build everything needed for the tf-model-analysis to run the model.
  Args:
    tf_transform_graph: A TFTransformOutput.
    schema: the schema of the input data.
  Returns:
    EvalInputReceiver function, which contains:

      - Tensorflow graph which parses raw untransformed features, applies the
        tf-transform preprocessing operators.
      - Set of raw, untransformed features.
      - Label against which predictions will be compared.
  """
  # Notice that the inputs are raw features, not transformed features here.
  raw_feature_spec = _get_raw_feature_spec(schema)

  serialized_tf_example = tf.compat.v1.placeholder(
      dtype=tf.string, shape=[None], name='input_example_tensor')

  # Add a parse_example operator to the tensorflow graph, which will parse
  # raw, untransformed, tf examples.
  features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)

  # Now that we have our raw examples, process them through the tf-transform
  # function computed during the preprocessing step.
  transformed_features = tf_transform_graph.transform_raw_features(
      features)

  # The key name MUST be 'examples'.
  receiver_tensors = {'examples': serialized_tf_example}

  # NOTE: Model is driven by transformed features (since training works on the
  # materialized output of TFT, but slicing will happen on raw features.
  features.update(transformed_features)

  return tfma.export.EvalInputReceiver(
      features=features,
      receiver_tensors=receiver_tensors,
      labels=transformed_features[_transformed_name(_LABEL_KEY)])


def _input_fn(file_pattern, data_accessor, tf_transform_output, batch_size=200):
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema)


# TFX will call this function
def run_fn(trainer_fn_args, schema):
  """Build the estimator using the high level API.
  Args:
    trainer_fn_args: Holds args used to train the model as name/value pairs.
    schema: Holds the schema of the training examples.
  Returns:
    A dict of the following:

      - estimator: The estimator that will be used for training and eval.
      - train_spec: Spec for training.
      - eval_spec: Spec for eval.
      - eval_input_receiver_fn: Input function for eval.
  """
  
  train_batch_size = 50
  eval_batch_size = 50

  tf_transform_graph = tft.TFTransformOutput(trainer_fn_args.transform_output)

  train_input_fn = lambda: _input_fn(  # pylint: disable=g-long-lambda
      trainer_fn_args.train_files,
      trainer_fn_args.data_accessor,
      tf_transform_graph,
      batch_size=train_batch_size)

  eval_input_fn = lambda: _input_fn(  # pylint: disable=g-long-lambda
      trainer_fn_args.eval_files,
      trainer_fn_args.data_accessor,
      tf_transform_graph,
      batch_size=eval_batch_size)

  train_spec = tf.estimator.TrainSpec(  # pylint: disable=g-long-lambda
      train_input_fn,
      max_steps=trainer_fn_args.train_steps)

  serving_receiver_fn = lambda: _example_serving_receiver_fn(  # pylint: disable=g-long-lambda
      tf_transform_graph, schema)

  exporter = tf.estimator.FinalExporter('forest-fires', serving_receiver_fn)
  eval_spec = tf.estimator.EvalSpec(
      eval_input_fn,
      steps=trainer_fn_args.eval_steps,
      exporters=[exporter],
      name='forest-fires-eval')

  run_config = tf.estimator.RunConfig(
      save_checkpoints_steps=499, keep_checkpoint_max=1)

  run_config = run_config.replace(model_dir=trainer_fn_args.serving_model_dir)

  estimator = _build_estimator(
      # Construct layers sizes with exponetial decay
      hidden_units=[32],
      config=run_config)

  # Create an input receiver for TFMA processing
  receiver_fn = lambda: _eval_input_receiver_fn(  # pylint: disable=g-long-lambda
      tf_transform_graph, schema)

  return {
      'estimator': estimator,
      'train_spec': train_spec,
      'eval_spec': eval_spec,
      'eval_input_receiver_fn': receiver_fn
  }
```
  